# Tratamento de dados - Secretaria de segurança do estado do Pará

## Descrição do notebook

O objetivo deste notebook é realizar o tratamento dos dados fornecidos pela Secretaria de Segurança Pública e Defesa Social do Estado do Pará para que possa ser analisado juntamente com os dados do estado paranaense.

Nesta versão, o arquivo utilizado é um CSV e para manipulação de dados é utilizado o pacote pyspark. 

**Observação:** O código contido nesse notebook foi baseado na solução desenvolvida por Mariana Kniss (https://acervodigital.ufpr.br/xmlui/handle/1884/85647), para validar o conteúdo de seu trabalho de conclusão de curso, intitulado "Análise de dados criminais sobre violência contra a mulher nos estados Pará e Paraná".

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Visão geral do conjunto de dados](#visao-geral-do-conjunto-de-dados)
5. [Filtros, tratamentos e padronização de valores](#filtros-tratamentos-e-padronizacao-de-valores)
6. [Ordenação das colunas](#ordernacao-das-colunas)
7. [Exportação dos dados formatados](#exportacao-dos-dados-formatados)

----


## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [ ]:
# pySpark (https://spark.apache.org/docs/latest/api/python/index.html)
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col, lower, when, substring, hour, to_timestamp, date_format, count, current_date, datediff, mean, median, variance, stddev, asc, expr, lit
from pyspark.sql.types import StringType, StructType, StructField, DateType
from pyspark.ml import stat
from pyspark.sql.window import Window

# Pandas (https://pandas.pydata.org/)
import pandas as pd

In [ ]:
# Criando sessão spark
spark = SparkSession.builder\
    .master('local[*]')\
    .appName("transformacao_VCM_PA")\
    .getOrCreate()

## Carregamento dos dados

In [ ]:
# Carregamento dos dados utilizando o arquivo CSV e spark
dados_csv = spark.read.csv("./dados/PA/raw_BD_VD_2010a2022_PA.csv",
                       sep = '@',
                       inferSchema= True,
                       header = True)

In [ ]:
# Exibe a quantidade de registros encontrados
dados_csv.count()

In [ ]:
# Exibe o tipo de variável
type(dados_csv)

In [ ]:
dados_csv.count()

In [ ]:
# Ordene o DataFrame pela coluna de índice em ordem descendente e selecione as primeiras linhas
tail_rows = dados_csv.orderBy(dados_csv["data_registro"].desc()).limit(10)

# Exiba as últimas linhas
tail_rows.show()

**Observação:** nota-se que há uma diferença de seis registros entre os dois métodos e arquivos de dados utilizados. Para se manter fiel à metodologia do TCC, optou-se para utilizar o CSV e spark, com um total de 196.256 registros. Todavia, conforme visto na célula acima, que simula o método tail(), esses seis registros adicionais não são registros de fato, pois os dados não estão coerentes. De qualquer forma, eles serão desconsiderados nas etapas de filtro.

----

In [ ]:
dados = dados_csv

## Visão geral do conjunto de dados <a id="visao-geral-do-conjunto-de-dados"></a>

In [ ]:
# Contagem do número de variáveis
v = len(dados.columns)
# Contagem do número de registros
n = dados.count()


print(f'O conjunto de dados possui {n} registros e {v} variáveis.')

In [ ]:
# Visualização do dataframe
dados.show()

In [ ]:
# Descrição dos tipos dos dados
dados.printSchema()

## Filtros, tratamentos e padronização de valores <a id="filtros-tratamentos-e-padronizacao-de-valores"></a>

### Filtros

#### Sexo das vítimas

In [ ]:
# Contagem dos valores distintos na coluna 'vit_sexo'
dados.groupBy("vit_sexo").count().orderBy("vit_sexo").show()

Filtrando apenas registros nos quais as vítimas são do sexo feminino.

In [ ]:
# Filtrando a base para que "vit_sexo" = "F"
dados_filtro_vit_F = dados.filter(dados.vit_sexo == "F")

# Exibindo a quantidade de registros restantes
n_feminino = dados_filtro_vit_F.count()

print(f'Foram retirados {n - n_feminino}, restando {n_feminino} registros válidos.')

#### Sexo dos autores

In [ ]:
# Contagem dos valores distintos na coluna 'aut_sexo'
dados.groupBy("aut_sexo").count().orderBy("aut_sexo").show()

Selecionando apenas regitros onde o sexo dos autores é diferente de feminino.

In [ ]:
# Filtrando a base para que "aut_sexo" diferente "F"
dados_filtro_aut_nF = dados_filtro_vit_F.filter(dados_filtro_vit_F.aut_sexo != "F")

# Exibindo o dataset VCM
n_autor_nf = dados_filtro_aut_nF.count()

print(f'Foram retirados {n_feminino - n_autor_nf}, restando {n_autor_nf} registros válidos.')

#### Ano do fato

A base de dados contava com registros de 2010 a 2023, porém foram selecionados apenas os casos ocorridos entre 2018 e 2022.

In [ ]:
# Filtrando a base para que ano registro >= 2018
dados_filtro_ano_registro = dados_filtro_aut_nF.filter(dados_filtro_aut_nF.ano_registro >= 2018)

# Exibindo o dataset VCM
n_registros_apos_2018 = dados_filtro_ano_registro.count()

print(f'Foram retirados {n_autor_nf - n_registros_apos_2018}, restando {n_registros_apos_2018} registros válidos.')

#### Especificação do fato

Para selecionar apenas os registros de violência contra mulher (VCM), foram mantidas apenas as linhas que contivesse uma das expressões abaixo na coluna 'especificacao_fato':

- "mulher"
- "penha"
- "feminicídio"
- "femicídio"
- "feminicidio"
- "femicidio"

In [ ]:
# Selecionando a base de VCM com base na variavel 'especializacao_fato', utilizando como filtro as palavras chave
palavras_chave = ["mulher", "feminicídio", "penha", "femicidio", "feminicidio", "femicídio"]

raw_VCM = dados_filtro_ano_registro.filter(lower(col("especializacao_fato")).like("%" + palavras_chave[0] + "%")
                                   | lower(col("especializacao_fato")).like("%" + palavras_chave[1] + "%")
                                   | lower(col("especializacao_fato")).like("%" + palavras_chave[2] + "%")
                                   | lower(col("especializacao_fato")).like("%" + palavras_chave[3] + "%"))

# Exibindo o dataset VCM
raw_VCM.show()

In [ ]:
n_vcm = raw_VCM.count()
print(f'Ao final, foram mantidos {n_vcm} registros. Ou seja, foram removidos {n_registros_apos_2018 - n_vcm} registros que não se encaixaram nos critérios definidos.')

In [ ]:
# Contagem dos valores distintos na coluna 'especializacao_fato'
raw_VCM.groupBy("especializacao_fato").count().orderBy("especializacao_fato").show()

### Tratamentos

#### Removação de atributos não relevantes para as análises

In [ ]:
# Remoção de colunas desnecessárias
colunas_para_remover = ["data_inst_proc", # Data de instauração do processo investigativo
						"data_concl_proc", # Data de conclusão do processo investigativo
						"sit_proc", # Situação do processo investigativo
						"grupo_ocorrencia", # Agrupamento (categorização) das ocorrências
						"sub_grupo", # Sub-agrupamento (categorização) das ocorrências
						"reg_integracao", # Regional de integração
						"risp", # RISP
						"aisp", # AISP
						"rua_fato", # Logradouro no qual aconteceu o fato
						"empresa", # Empresa de transporte relacionada ao fato
						"linha", # Linha de transporte relacionada ao fato
						"tipo_transporte", # Tipo de transporte relacionada ao fato
						"complemento", # Complemento do endereço
						"atuacao", # Atuação - Todos os valores constam como 'VITIMA'
						"meio_locomocao", # Meio de locomoção do autor
						"cor_veiculo", # Cor do veículo do autor
						"marca_veic_fuga", # Marca do veículo do autor
						"modelo_do_veic_fuga", # Modelo do veículo do autor
						"relatorio", # Relatório
						"qtd_autor", # Quantidade de autores relacionados à ocorrência
						"ident_autoria" # Se a identidade do autor é conhecida
						]

raw_VCM = raw_VCM.drop(*colunas_para_remover)

In [ ]:
# Contagem do número de variáveis
v_raw_vcm = len(raw_VCM.columns)
# Contagem do número de registros
n_raw_vcm = raw_VCM.count()

print(f'O conjunto de dados possui {n_raw_vcm} registros e {v_raw_vcm} variáveis.')

In [ ]:
# Descrição do tipo dos dados
raw_VCM.printSchema()

In [ ]:
# Visualização do dataframe de VCM
raw_VCM.show()

#### Alteração dos tipos de variáveis

In [ ]:
# Alteração do tipo da coluna 'data_registro', de string para timestamp
dados_VCM = raw_VCM.withColumn("data_registro", to_timestamp(col("data_registro")))

# Seleção e alteração a data das variaveis
dados_VCM = dados_VCM.withColumn("data_registro", when(col("data_registro").isNotNull(), col("data_registro").cast(DateType())))
dados_VCM = dados_VCM.withColumn("data_fato", when(col("data_fato").isNotNull(), col("data_fato").cast(DateType())))
dados_VCM = dados_VCM.withColumn("vit_dt_nasc", when(col("vit_dt_nasc").isNotNull(), col("vit_dt_nasc").cast(DateType())))
dados_VCM = dados_VCM.withColumn("aut_data_nasc", when(col("aut_data_nasc").isNotNull(), col("aut_data_nasc").cast(DateType())))

In [ ]:
# Extraindo somente a parte da hora das variáveis, tipadas como string após a alteração
dados_VCM = dados_VCM.withColumn("hora_registro", col("hora_registro").substr(12, 8))
dados_VCM = dados_VCM.withColumn("hora_fato", col("hora_fato").substr(12, 8))

#### Adição de uma nova coluna referente à idade da vítima

In [ ]:
# Use a função when para criar uma nova coluna 'vit_idade' com registros nulos
dados_VCM = dados_VCM.withColumn(
    'vit_idade',
    when(col('vit_idade') > 105, None).otherwise(col('vit_idade'))
)

In [ ]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM = dados_VCM.withColumn("vit_dt_nasc", when(dados_VCM["vit_dt_nasc"] <= "1900-09-09", None).otherwise(dados_VCM["vit_dt_nasc"]))

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM = dados_VCM.withColumn("vit_idade_calculada",
                   when(
                       (dados_VCM["data_fato"].isNotNull()) & (dados_VCM["vit_dt_nasc"].isNotNull()),
                       (datediff(dados_VCM["data_fato"], dados_VCM["vit_dt_nasc"]) / 365).cast("int")
                   ).otherwise(None))

#### Idade do autor

In [ ]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM = dados_VCM.withColumn("aut_data_nasc", when(dados_VCM["aut_data_nasc"] <= "1900-09-09", None).otherwise(dados_VCM["aut_data_nasc"]))

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM = dados_VCM.withColumn("aut_idade_calculada",
                   when(
                       (dados_VCM["data_fato"].isNotNull()) & (dados_VCM["aut_data_nasc"].isNotNull()),
                       (datediff(dados_VCM["data_fato"], dados_VCM["aut_data_nasc"]) / 365).cast("int")
                   ).otherwise(None))

In [ ]:
# Use a função 'when' para criar uma nova coluna com as faixas etárias
dados_VCM = dados_VCM.withColumn(
    "vit_faixa_etaria_calculada",
    when((dados_VCM.vit_idade_calculada >= 0) & (dados_VCM.vit_idade_calculada <= 11), "CRIANCA (0 A 11 ANOS)")
    .when((dados_VCM.vit_idade_calculada >= 12) & (dados_VCM.vit_idade_calculada <= 17), "ADOLESCENTE (12 A 17 ANOS)")
    .when((dados_VCM.vit_idade_calculada >= 18) & (dados_VCM.vit_idade_calculada <= 24), "ADULTO I (18 A 24 ANOS)")
    .when((dados_VCM.vit_idade_calculada >= 25) & (dados_VCM.vit_idade_calculada <= 29), "ADULTO II (25 A 29 ANOS)")
    .when((dados_VCM.vit_idade_calculada >= 30) & (dados_VCM.vit_idade_calculada <= 34), "ADULTO III (30 A 34 ANOS)")
    .when((dados_VCM.vit_idade_calculada >= 35) & (dados_VCM.vit_idade_calculada <= 64), "ADULTO IV (35 A 64 ANOS)")
    .when(dados_VCM.vit_idade_calculada >= 65, "IDOSO")
    .otherwise("Desconhecida")  # Lidando com vit_idade_calculadas fora das faixas
)

In [ ]:
# Use a função 'when' para criar uma nova coluna com as faixas etárias
dados_VCM = dados_VCM.withColumn(
    "aut_faixa_etaria_calculada",
    when((dados_VCM.aut_idade_calculada >= 0) & (dados_VCM.aut_idade_calculada <= 11), "CRIANCA (0 A 11 ANOS)")
    .when((dados_VCM.aut_idade_calculada >= 12) & (dados_VCM.aut_idade_calculada <= 17), "ADOLESCENTE (12 A 17 ANOS)")
    .when((dados_VCM.aut_idade_calculada >= 18) & (dados_VCM.aut_idade_calculada <= 24), "ADULTO I (18 A 24 ANOS)")
    .when((dados_VCM.aut_idade_calculada >= 25) & (dados_VCM.aut_idade_calculada <= 29), "ADULTO II (25 A 29 ANOS)")
    .when((dados_VCM.aut_idade_calculada >= 30) & (dados_VCM.aut_idade_calculada <= 34), "ADULTO III (30 A 34 ANOS)")
    .when((dados_VCM.aut_idade_calculada >= 35) & (dados_VCM.aut_idade_calculada <= 64), "ADULTO IV (35 A 64 ANOS)")
    .when(dados_VCM.aut_idade_calculada >= 65, "IDOSO")
    .otherwise("Desconhecida")  # Lidando com aut_idade_calculadas fora das faixas
)

In [ ]:
# Visualizando o dataframe atualizado
dados_VCM.show()

In [ ]:
# Verifica a quantidade de casos nos quais as vítimas tinham mais de 18 anos
dados_VCM_maiores_18_PA = dados_VCM.filter(dados_VCM['vit_idade_calculada'] >= 18)
dados_VCM_maiores_18_PA.count()

In [ ]:
# Verifica a quantidade total de registros
dados_VCM.count()

In [ ]:
# Cria um novo dataframe para ser salvo referente aos dados do estado do Pará
dados_VCM_PA = dados_VCM
# Visualização do dataframe
dados_VCM_PA.show()

#### Tratamento dos meses

In [ ]:
# Dicionário de mapeamento de meses para números
meses = {
    "JANEIRO": 1,
    "FEVEREIRO": 2,
    "MARCO": 3,
    "ABRIL": 4,
    "MAIO": 5,
    "JUNHO": 6,
    "JULHO": 7,
    "AGOSTO": 8,
    "SETEMBRO": 9,
    "OUTUBRO": 10,
    "NOVEMBRO": 11,
    "DEZEMBRO": 12
}

# Loop para criar a coluna "mes_registro" para cada mês
for mes, numero in meses.items():
    dados_VCM_PA = dados_VCM_PA.withColumn("mes_registro", f.when(dados_VCM_PA.mes_registro == mes, numero).otherwise(dados_VCM_PA.mes_registro))

# Loop para criar a coluna "mes_registro" para cada mês
for mes, numero in meses.items():
    dados_VCM_PA = dados_VCM_PA.withColumn("mes_fato", f.when(dados_VCM_PA.mes_registro == mes, numero).otherwise(dados_VCM_PA.mes_registro))

#### Faixa horária

In [ ]:
# Função 'when' para criar a coluna 'faixa_horaria' com base na coluna 'hora_fato'
dados_VCM_PA = dados_VCM_PA.withColumn("faixa_horaria",
                                       when((dados_VCM_PA['hora_fato'] >= '06:00:00') & (dados_VCM_PA['hora_fato'] < '12:00:00'), 'Manhã')
                                      .when((dados_VCM_PA['hora_fato'] >= '12:00:00') & (dados_VCM_PA['hora_fato'] < '18:00:00'), 'Tarde')
                                      .when((dados_VCM_PA['hora_fato'] >= '18:00:00') & (dados_VCM_PA['hora_fato'] < '23:59:59'), 'Noite')
                                      .when((dados_VCM_PA['hora_fato'] >= '00:00:00') & (dados_VCM_PA['hora_fato'] < '06:00:00'), 'Madrugada')
                                      .otherwise('Outro'))


# Função 'when' para criar a coluna 'faixa_horaria_6h' com base na coluna 'hora_fato'
dados_VCM_PA = dados_VCM_PA.withColumn("faixa_horaria_6h",
                                       when((dados_VCM_PA['hora_fato'] >= '06:00:00') & (dados_VCM_PA['hora_fato'] < '12:00:00'), '06h - 12h')
                                      .when((dados_VCM_PA['hora_fato'] >= '12:00:00') & (dados_VCM_PA['hora_fato'] < '18:00:00'), '12h - 18h')
                                      .when((dados_VCM_PA['hora_fato'] >= '18:00:00') & (dados_VCM_PA['hora_fato'] < '23:59:59'), '18h - 00h')
                                      .when((dados_VCM_PA['hora_fato'] >= '00:00:00') & (dados_VCM_PA['hora_fato'] < '06:00:00'), '00h - 06h')
                                      .otherwise('Outro'))

# Função 'when' para criar a coluna 'faixa_horaria_2h' com base na coluna 'hora_fato'
dados_VCM_PA = dados_VCM_PA.withColumn("faixa_horaria_2h",
                                       when((dados_VCM_PA['hora_fato'] >= '06:00:00') & (dados_VCM_PA['hora_fato'] < '08:00:00'), '06h - 08h')
                                      .when((dados_VCM_PA['hora_fato'] >= '08:00:00') & (dados_VCM_PA['hora_fato'] < '10:00:00'), '08h - 10h')
                                      .when((dados_VCM_PA['hora_fato'] >= '10:00:00') & (dados_VCM_PA['hora_fato'] < '12:00:00'), '10h - 12h')
                                      .when((dados_VCM_PA['hora_fato'] >= '12:00:00') & (dados_VCM_PA['hora_fato'] < '14:00:00'), '12h - 14h')
                                      .when((dados_VCM_PA['hora_fato'] >= '14:00:00') & (dados_VCM_PA['hora_fato'] < '16:00:00'), '14h - 16h')
                                      .when((dados_VCM_PA['hora_fato'] >= '16:00:00') & (dados_VCM_PA['hora_fato'] < '18:00:00'), '16h - 18h')
                                      .when((dados_VCM_PA['hora_fato'] >= '18:00:00') & (dados_VCM_PA['hora_fato'] < '20:00:00'), '18h - 20h')
                                      .when((dados_VCM_PA['hora_fato'] >= '20:00:00') & (dados_VCM_PA['hora_fato'] < '22:00:00'), '20h - 22h')
                                      .when((dados_VCM_PA['hora_fato'] >= '22:00:00') & (dados_VCM_PA['hora_fato'] < '23:59:59'), '22h - 00h')
                                      .when((dados_VCM_PA['hora_fato'] >= '00:00:00') & (dados_VCM_PA['hora_fato'] < '02:00:00'), '00h - 02h')
                                      .when((dados_VCM_PA['hora_fato'] >= '02:00:00') & (dados_VCM_PA['hora_fato'] < '04:00:00'), '02h - 04h')
                                      .when((dados_VCM_PA['hora_fato'] >= '04:00:00') & (dados_VCM_PA['hora_fato'] < '06:00:00'), '04h - 06h')
                                      .otherwise('Outro'))

dados_VCM_PA.show()

In [ ]:
# Remoção de mais colunas
colunas_remover = ['fx_4_hor',
					'fx_12_hr',
					'latitude',
					'longitude',
					'distrito',
					'regionais',
					'vit_idade',
					'vit_fx_etaria',
					'vit_sexo',
					'vit_situacao_emprego',
					'aut_idade',
					'aut_fx_etaria'
				]
dados_VCM_PA = dados_VCM_PA.drop(*colunas_remover)

In [ ]:
# Contagem do número de variáveis
v_vcm_pa = len(dados_VCM_PA.columns)
# Contagem do número de registros
n_vcm_pa = dados_VCM_PA.count()

print(f'O conjunto de dados possui {n_vcm_pa} registros e {v_vcm_pa} variáveis.')

## Ordenação das colunas <a id="ordernacao-das-colunas"></a>

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA.columns
colunas_reorganizadas = colunas[:3] + [colunas[4]] + [colunas[3]] + colunas[5:]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PA_ordenadas.show()

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA_ordenadas.columns
colunas_reorganizadas = colunas[:5] + [colunas[41]] + colunas[5:41]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA_ordenadas.select(colunas_reorganizadas)

# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA_ordenadas.columns
colunas_reorganizadas = colunas[:6] + [colunas[41]] + colunas[6:41]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA_ordenadas.select(colunas_reorganizadas)

dados_VCM_PA = dados_VCM_PA_ordenadas
dados_VCM_PA.show()

In [ ]:
dados_VCM_PA = dados_VCM_PA \
    .withColumnRenamed("vit_idade_calculada", "vit_idade") \
    .withColumnRenamed("aut_idade_calculada", "aut_idade") \
    .withColumnRenamed("vit_faixa_etaria_calculada", "vit_faixa_etaria") \
    .withColumnRenamed("aut_faixa_etaria_calculada", "aut_faixa_etaria")

In [ ]:
dados_VCM_PA.show()

In [ ]:
dados_VCM_PA.select('identificacao_fato').distinct().count()
dados_VCM_PA = dados_VCM_PA.withColumn("estado", lit("PA"))
dados_VCM_PA.show()

In [ ]:
colunas_remover = ['consolidado', 'fato_real', 'meio_emp_deac', 'causa_presumivel', 'especializacao_fato', 'identificacao_fato', 'vit_estado_civil', 'aut_sexo', 'aut_sit_emprego', 'aut_est_civil']
dados_VCM_PA = dados_VCM_PA.drop(*colunas_remover)

len(dados_VCM_PA.columns)

In [ ]:
dados_VCM_PA = dados_VCM_PA \
    .withColumnRenamed("meio_empregado_sisp", "meio_empregado") \
    .withColumnRenamed("municipios", "municipio") \
    .withColumnRenamed("bairros", "bairro") \
    .withColumnRenamed("aut_data_nasc", "aut_dt_nasc")

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA.columns
colunas_reorganizadas = colunas[:8] + [colunas[9]] + [colunas[10]]+ [colunas[11]]+ [colunas[12]]+ [colunas[8]] + colunas[13:]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA.select(colunas_reorganizadas)

# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA_ordenadas.columns
colunas_reorganizadas = colunas[:12] + [colunas[15]] + [colunas[16]]+ [colunas[17]]+ [colunas[14]]+ [colunas[12]]+ [colunas[13]]+ colunas[18:]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA.select(colunas_reorganizadas)

# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA_ordenadas.columns
colunas_reorganizadas = colunas[:19] + [colunas[27]] + [colunas[29]]+ colunas[19:27]+ [colunas[28]]+  colunas[30:]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA.select(colunas_reorganizadas)

# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PA_ordenadas.columns
colunas_reorganizadas = colunas[:25] + [colunas[29]] + [colunas[30]]+ [colunas[26]]+ colunas[27:29]+ [colunas[25]]+  colunas[31:]

# Selecione as colunas na ordem desejada
dados_VCM_PA_ordenadas = dados_VCM_PA.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PA_ordenadas.show()

In [ ]:
len(dados_VCM_PA_ordenadas.columns)

In [ ]:
dados_VCM_PA = dados_VCM_PA_ordenadas
dados_VCM_PA.show()

In [ ]:
dados_VCM_PA.printSchema()

In [ ]:
dados_VCM_PA.count()

## Exportação dos dados formatados <a id="exportacao-dos-dados-formatados"></a>

In [ ]:
import datetime as dt # Manipulação de data

dados_VCM_pd = dados_VCM_PA.toPandas()

#dados_VCM_pd.to_csv('./dados-tratados/' + dt.datetime.now().strftime('%Y%m%d%H%M%S') + '-dados_VCM_PA.csv', index=False, sep='|', encoding='utf-8')
dados_VCM_pd.to_csv('./dados-tratados/dados_VCM_PA.csv', index=False, sep='|', encoding='utf-8')